# Latin Hypercube Sampling for Experimental Design

This application helps researchers design experiments using Latin Hypercube Sampling (LHS). It's particularly useful for material science applications like perovskite solar cells where multiple variables need to be explored efficiently.

## Features
- Define continuous and categorical variables with their ranges
- Apply weighted sampling to focus on regions of interest
- Visualize the experimental design space
- Export experiment plans to CSV
- Save and load variable configurations

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import qmc
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
import os
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from mpl_toolkits.mplot3d import Axes3D
import io
import base64
import warnings  
# Suppress warnings  
warnings.filterwarnings("ignore")

# Set style for plots
plt.style.use('default')
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.2)

In [2]:
class Variable:
    """Class to represent an experimental variable"""
    
    def __init__(self, name, var_type, min_val=None, max_val=None, step=None, categories=None, weight=1.0):
        self.name = name
        self.var_type = var_type  # 'continuous', 'discrete', or 'categorical'
        
        # Validate inputs based on variable type
        if var_type in ['continuous', 'discrete']:
            if min_val is None or max_val is None:
                raise ValueError(f"Min and max values must be provided for {var_type} variables")
            if min_val >= max_val:
                raise ValueError(f"Min value must be less than max value for {var_type} variables")
            
            if var_type == 'discrete' and step is not None and step <= 0:
                raise ValueError("Step must be positive for discrete variables")
        
        if var_type == 'categorical' and (categories is None or len(categories) == 0):
            raise ValueError("Categories must be provided for categorical variables")
        
        self.min_val = min_val
        self.max_val = max_val
        self.step = step
        self.categories = categories if categories else []
        self.weight = weight  # For weighted sampling (1.0 = normal weight)
        
    def to_dict(self):
        """Convert variable to dictionary for saving"""
        return {
            'name': self.name,
            'var_type': self.var_type,
            'min_val': self.min_val,
            'max_val': self.max_val,
            'step': self.step,
            'categories': self.categories,
            'weight': self.weight
        }
    
    @classmethod
    def from_dict(cls, data):
        """Create variable from dictionary"""
        return cls(
            name=data['name'],
            var_type=data['var_type'],
            min_val=data['min_val'],
            max_val=data['max_val'],
            step=data['step'],
            categories=data['categories'],
            weight=data['weight']
        )

In [3]:
class LHSExperimentalDesign:  
    """Latin Hypercube Sampling for Experimental Design"""  
      
    def __init__(self):  
        """Initialize the LHS experimental design"""  
        self.variables = []  
        self.experiment_df = None  
        self.n_samples = 0  
        self.seed = None  
        self.metadata = {}  
      
    def add_variable(self, variable):  
        """Add a variable to the experimental design"""  
        if not isinstance(variable, Variable):  
            raise TypeError("Variable must be an instance of the Variable class")  
          
        # Check if variable with same name already exists  
        if any(v.name == variable.name for v in self.variables):  
            raise ValueError(f"Variable with name '{variable.name}' already exists")  
          
        self.variables.append(variable)  
      
    def remove_variable(self, variable_name):  
        """Remove a variable from the experimental design"""  
        self.variables = [v for v in self.variables if v.name != variable_name]  
          
        # If we have generated samples, regenerate without this variable  
        if self.experiment_df is not None:  
            self._regenerate_samples()  
      
    def generate_samples(self, n_samples, seed=None):  
        """Generate Latin Hypercube samples for all variables"""  
        if n_samples <= 0:  
            raise ValueError("Number of samples must be positive")  
          
        if len(self.variables) == 0:  
            raise ValueError("No variables defined. Add variables before generating samples.")  
          
        self.n_samples = n_samples  
        self.seed = seed  
          
        # Create a dataframe to store the samples  
        self.experiment_df = pd.DataFrame()  
          
        # Generate samples for each variable  
        for var in self.variables:  
            if var.var_type == 'continuous':  
                samples = self._generate_continuous_samples(var, n_samples, seed)  
            elif var.var_type == 'discrete':  
                samples = self._generate_discrete_samples(var, n_samples, seed)  
            elif var.var_type == 'categorical':  
                samples = self._generate_categorical_samples(var, n_samples, seed)  
            else:  
                raise ValueError(f"Unknown variable type: {var.var_type}")  
              
            self.experiment_df[var.name] = samples  
          
        return self.experiment_df  
      
    def _generate_continuous_samples(self, variable, n_samples, seed=None):  
        """Generate Latin Hypercube samples for a continuous variable"""  
        np.random.seed(seed)  
          
        # Generate uniform samples in [0, 1]  
        u = np.random.random(n_samples)  
          
        # Generate stratified samples  
        stratified = (np.arange(n_samples) + u) / n_samples  
          
        # Apply variable weight (higher weight = more samples in the center)  
        if variable.weight != 1.0:  
            # Transform to make distribution more centered  
            stratified = self._apply_weight(stratified, variable.weight)  
          
        # Scale to the variable range  
        samples = variable.min_val + stratified * (variable.max_val - variable.min_val)  
          
        return samples  
      
    def _generate_discrete_samples(self, variable, n_samples, seed=None):  
        """Generate Latin Hypercube samples for a discrete variable"""  
        np.random.seed(seed)  
          
        # Generate uniform samples in [0, 1]  
        u = np.random.random(n_samples)  
          
        # Generate stratified samples  
        stratified = (np.arange(n_samples) + u) / n_samples  
          
        # Apply variable weight  
        if variable.weight != 1.0:  
            stratified = self._apply_weight(stratified, variable.weight)  
          
        # Calculate the number of possible values  
        if variable.step <= 0:  
            raise ValueError("Step size must be positive for discrete variables")  
          
        n_values = int((variable.max_val - variable.min_val) / variable.step) + 1  
          
        # Scale to the variable range and round to the nearest valid value  
        scaled = variable.min_val + stratified * (variable.max_val - variable.min_val)  
        samples = np.round((scaled - variable.min_val) / variable.step) * variable.step + variable.min_val  
          
        # Ensure we don't exceed the max value due to rounding  
        samples = np.minimum(samples, variable.max_val)  
          
        return samples  
      
    def _generate_categorical_samples(self, variable, n_samples, seed=None):  
        """Generate Latin Hypercube samples for a categorical variable"""  
        if seed is not None:  
            np.random.seed(seed)  
          
        if not variable.categories:  
            raise ValueError(f"No categories defined for variable {variable.name}")  
          
        # For categorical variables, we need to ensure balanced sampling  
        n_categories = len(variable.categories)  
          
        # Calculate how many samples per category  
        samples_per_category = n_samples // n_categories  
        remainder = n_samples % n_categories  
          
        # Create a list of category indices  
        category_indices = []  
        for i in range(n_categories):  
            # Add extra samples for the first 'remainder' categories  
            n_cat_samples = samples_per_category + (1 if i < remainder else 0)  
            category_indices.extend([i] * n_cat_samples)  
          
        # Shuffle the indices  
        np.random.shuffle(category_indices)  
          
        # Convert indices to category values  
        samples = [variable.categories[i] for i in category_indices]  
          
        return samples  
      
    def _apply_weight(self, samples, weight):  
        """Apply a weight to the samples to control their distribution"""  
        # Weight > 1 means more samples in the center  
        # Weight < 1 means more samples at the extremes  
          
        # Transform to [-1, 1] range  
        transformed = 2 * samples - 1  
          
        # Apply the weight (using a power function)  
        if weight != 0:  # Avoid division by zero  
            weighted = np.sign(transformed) * np.abs(transformed) ** (1 / weight)  
        else:  
            # For weight=0, we'd get all samples at the extremes  
            weighted = np.sign(transformed)  
          
        # Transform back to [0, 1] range  
        return (weighted + 1) / 2  
      
    def _regenerate_samples(self):  
        """Regenerate samples with the current variables"""  
        if self.n_samples > 0:  
            self.generate_samples(self.n_samples, self.seed)  
      
    def plot_distributions(self):  
        """Plot histograms of the variable distributions"""  
        import numpy as np  # Import numpy at the beginning of the method  
          
        if self.experiment_df is None:  
            return None  
          
        # Get continuous and discrete variables for plotting  
        plot_vars = [v for v in self.variables if v.var_type in ['continuous', 'discrete']]  
        n_plot_vars = len(plot_vars)  
          
        if n_plot_vars == 0:  
            return None  
          
        # Create histograms  
        fig, axes = plt.subplots(1, n_plot_vars, figsize=(4*n_plot_vars, 4))  
        if n_plot_vars == 1:  
            axes = [axes]  # Make it iterable  
          
        for i, var in enumerate(plot_vars):  
            # Replace inf values with NaN to avoid the warning  
            data = self.experiment_df[var.name].copy()  
            data.replace([np.inf, -np.inf], np.nan, inplace=True)  
              
            # Use matplotlib's hist instead of seaborn to avoid warnings  
            axes[i].hist(data, bins=10, alpha=0.7, density=True)  
              
            # Add a density curve  
            if len(data.dropna()) > 1:  # Need at least 2 points for kde  
                from scipy import stats  
                x = np.linspace(data.min(), data.max(), 100)  
                kde = stats.gaussian_kde(data.dropna())  
                axes[i].plot(x, kde(x), 'r-')  
              
            axes[i].set_xlabel(var.name)  
            axes[i].set_ylabel('Density')  
              
            # Add weight information to the title  
            if var.weight != 1.0:  
                weight_info = f" (weight={var.weight:.1f})"  
            else:  
                weight_info = ""  
            axes[i].set_title(f"Distribution{weight_info}")  
          
        plt.tight_layout()  
        return fig
      
    def plot_samples(self, max_plots=3):  
        """Plot the generated samples"""  
        if self.experiment_df is None or len(self.variables) < 2:  
            return None  
          
        # Get continuous and discrete variables for plotting  
        plot_vars = [v for v in self.variables if v.var_type in ['continuous', 'discrete']]  
        n_plot_vars = len(plot_vars)  
          
        if n_plot_vars < 2:  
            # Need at least 2 continuous/discrete variables for plotting  
            return None  
          
        # Limit the number of variables to plot  
        plot_vars = plot_vars[:max_plots]  
        n_plot_vars = len(plot_vars)  
          
        # Create pairwise scatter plots  
        fig = plt.figure(figsize=(12, 10))  
          
        # If we have exactly 3 variables, create a 3D plot  
        if n_plot_vars == 3:  
            ax = fig.add_subplot(111, projection='3d')  
            ax.scatter(  
                self.experiment_df[plot_vars[0].name],  
                self.experiment_df[plot_vars[1].name],  
                self.experiment_df[plot_vars[2].name],  
                s=50, alpha=0.8  
            )  
            ax.set_xlabel(plot_vars[0].name)  
            ax.set_ylabel(plot_vars[1].name)  
            ax.set_zlabel(plot_vars[2].name)  
            ax.set_title("3D Visualization of Experimental Design")  
        else:  
            # Handle the case when n_pairs is 0 (exactly 2 variables)  
            if n_plot_vars == 2:  
                ax = fig.add_subplot(111)  
                ax.scatter(  
                    self.experiment_df[plot_vars[0].name],  
                    self.experiment_df[plot_vars[1].name],  
                    s=50, alpha=0.8  
                )  
                ax.set_xlabel(plot_vars[0].name)  
                ax.set_ylabel(plot_vars[1].name)  
                ax.set_title("Experimental Design Space")  
            else:  
                # Create a grid of pairwise plots  
                n_pairs = n_plot_vars * (n_plot_vars - 1) // 2  
                rows = max(1, int(np.ceil(n_pairs / 2)))  
                cols = min(2, n_pairs)  
                  
                plot_idx = 1  
                for i in range(n_plot_vars):  
                    for j in range(i+1, n_plot_vars):  
                        ax = fig.add_subplot(rows, cols, plot_idx)  
                        ax.scatter(  
                            self.experiment_df[plot_vars[i].name],  
                            self.experiment_df[plot_vars[j].name],  
                            s=50, alpha=0.8  
                        )  
                        ax.set_xlabel(plot_vars[i].name)  
                        ax.set_ylabel(plot_vars[j].name)  
                        plot_idx += 1  
          
        plt.tight_layout()  
        return fig  
      
    def plot_parallel_coordinates(self):  
        """Plot parallel coordinates visualization of the samples"""  
        if self.experiment_df is None or len(self.variables) < 2:  
            return None  
          
        # Get continuous and discrete variables for plotting  
        plot_vars = [v for v in self.variables if v.var_type in ['continuous', 'discrete']]  
        if len(plot_vars) < 2:  
            return None  
          
        # Create a copy of the dataframe with only the variables we want to plot  
        plot_df = self.experiment_df[[v.name for v in plot_vars]].copy()  
          
        # Normalize the data for better visualization  
        for var in plot_vars:  
            if var.var_type in ['continuous', 'discrete']:  
                min_val = plot_df[var.name].min()  
                max_val = plot_df[var.name].max()  
                if max_val > min_val:  # Avoid division by zero  
                    plot_df[var.name] = (plot_df[var.name] - min_val) / (max_val - min_val)  
          
        # Create the plot  
        fig, ax = plt.subplots(figsize=(10, 6))  
        pd.plotting.parallel_coordinates(plot_df, class_column=plot_vars[0].name, ax=ax, alpha=0.5)  
          
        # Improve the plot appearance  
        ax.set_title("Parallel Coordinates Plot of Experimental Design")  
        ax.grid(True)  
          
        plt.tight_layout()  
        return fig  
      
    def save_experiment(self, filename):  
        """Save the experimental design to a file"""
        if self.experiment_df is None:  
            raise ValueError("No samples generated yet. Generate samples before saving.")  
          
        try:  
            # Create a dictionary with all the experiment data  
            experiment_data = {  
                'variables': [v.__dict__ for v in self.variables],  
                'n_samples': self.n_samples,  
                'seed': self.seed,  
                'metadata': self.metadata,  
                'experiment_df': self.experiment_df.to_dict(orient='list')  
            }  
              
            # Save to file  
            with open(filename, 'w') as f:  
                json.dump(experiment_data, f, indent=2)  
              
            return True  
        except Exception as e:  
            print(f"Error saving experiment: {str(e)}")  
            return False  
      
    def load_experiment(self, filename):  
        """Load an experimental design from a file""" 
        try:  
            # Load from file  
            with open(filename, 'r') as f:  
                experiment_data = json.load(f)  
              
            # Recreate the variables  
            self.variables = []  
            for var_dict in experiment_data['variables']:  
                var = Variable(  
                    name=var_dict['name'],  
                    var_type=var_dict['var_type'],  
                    min_val=var_dict.get('min_val'),  
                    max_val=var_dict.get('max_val'),  
                    step=var_dict.get('step'),  
                    categories=var_dict.get('categories'),  
                    weight=var_dict.get('weight', 1.0)  
                )  
                self.variables.append(var)  
              
            # Recreate the experiment dataframe  
            self.experiment_df = pd.DataFrame(experiment_data['experiment_df'])  
              
            # Set other properties  
            self.n_samples = experiment_data['n_samples']  
            self.seed = experiment_data['seed']  
            self.metadata = experiment_data.get('metadata', {})  
              
            return True  
        except Exception as e:  
            print(f"Error loading experiment: {str(e)}")  
            return False  
      
    def export_to_csv(self, filename):  
        """Export the experimental design to a CSV file"""  
        if self.experiment_df is None:  
            raise ValueError("No samples generated yet. Generate samples before exporting.")  
          
        try:  
            self.experiment_df.to_csv(filename, index=False)  
            return True  
        except Exception as e:  
            print(f"Error exporting to CSV: {str(e)}")  
            return False  
      
    def get_experiment_summary(self):  
        """Get a summary of the experimental design"""  
        if self.experiment_df is None:  
            return "No samples generated yet."  
          
        summary = f"Experimental Design Summary:\\n"  
        summary += f"Number of variables: {len(self.variables)}\\n"  
        summary += f"Number of samples: {self.n_samples}\\n"  
        summary += f"Random seed: {self.seed}\\n\\n"  
          
        summary += "Variables:\\n"  
        for var in self.variables:  
            summary += f"- {var.name} ({var.var_type})"  
            if var.var_type in ['continuous', 'discrete']:  
                summary += f": [{var.min_val}, {var.max_val}]"  
                if var.var_type == 'discrete':  
                    summary += f", step={var.step}"  
            elif var.var_type == 'categorical':  
                summary += f": {var.categories}"  
              
            if var.weight != 1.0:  
                summary += f", weight={var.weight}"  
              
            summary += "\\n"  
          
        return summary  
      
    def get_experiment_html(self):  
        """Get an HTML representation of the experimental design"""  
        if self.experiment_df is None:  
            return "<p>No samples generated yet.</p>"  
          
        html = "<h3>Experimental Design</h3>"  
        html += f"<p><b>Number of variables:</b> {len(self.variables)}</p>"  
        html += f"<p><b>Number of samples:</b> {self.n_samples}</p>"  
        html += f"<p><b>Random seed:</b> {self.seed}</p>"  
          
        html += "<h4>Variables</h4>"  
        html += "<ul>"  
        for var in self.variables:  
            html += f"<li><b>{var.name}</b> ({var.var_type})"  
            if var.var_type in ['continuous', 'discrete']:  
                html += f": [{var.min_val}, {var.max_val}]"  
                if var.var_type == 'discrete':  
                    html += f", step={var.step}"  
            elif var.var_type == 'categorical':  
                html += f": {var.categories}"  
              
            if var.weight != 1.0:  
                html += f", weight={var.weight}"  
              
            html += "</li>"  
        html += "</ul>"  
          
        # Add a preview of the data  
        html += "<h4>Data Preview</h4>"  
        html += self.experiment_df.head().to_html()  
          
        return html  
      
    def get_experiment_svg(self):  
        """Get an SVG visualization of the experimental design"""  
        if self.experiment_df is None:  
            return None  
          
        # Create a figure with the plots  
        fig = plt.figure(figsize=(12, 8))  
          
        # Add a title  
        fig.suptitle("Experimental Design Visualization", fontsize=16)  
          
        # Create a grid for the plots  
        gs = gridspec.GridSpec(2, 2, figure=fig)  
          
        # Plot the samples  
        ax1 = fig.add_subplot(gs[0, 0])  
        ax1.set_title("Sample Distribution")  
          
        # Get the first two continuous/discrete variables  
        plot_vars = [v for v in self.variables if v.var_type in ['continuous', 'discrete']]  
        if len(plot_vars) >= 2:  
            ax1.scatter(  
                self.experiment_df[plot_vars[0].name],  
                self.experiment_df[plot_vars[1].name],  
                s=50, alpha=0.8  
            )  
            ax1.set_xlabel(plot_vars[0].name)  
            ax1.set_ylabel(plot_vars[1].name)  
        else:  
            ax1.text(0.5, 0.5, "Not enough continuous/discrete variables",   
                    ha='center', va='center', transform=ax1.transAxes)  
          
        # Plot the distributions  
        ax2 = fig.add_subplot(gs[0, 1])  
        ax2.set_title("Variable Distributions")  
          
        if len(plot_vars) >= 1:  
            # Replace inf values with NaN to avoid the warning  
            data = self.experiment_df[plot_vars[0].name].copy()  
            data.replace([np.inf, -np.inf], np.nan, inplace=True)  
              
            # Use matplotlib's hist instead of seaborn  
            ax2.hist(data, bins=10, alpha=0.7, density=True)  
            ax2.set_xlabel(plot_vars[0].name)  
            ax2.set_ylabel("Density")  
        else:  
            ax2.text(0.5, 0.5, "No continuous/discrete variables",   
                    ha='center', va='center', transform=ax2.transAxes)  
          
        # Add experiment info  
        ax3 = fig.add_subplot(gs[1, :])  
        ax3.axis('off')  
        info_text = f"Experimental Design Summary:\\n"  
        info_text += f"Number of variables: {len(self.variables)}\\n"  
        info_text += f"Number of samples: {self.n_samples}\\n"  
        info_text += f"Random seed: {self.seed}\\n\\n"  
          
        info_text += "Variables:\\n"  
        for var in self.variables:  
            info_text += f"- {var.name} ({var.var_type})"  
            if var.var_type in ['continuous', 'discrete']:  
                info_text += f": [{var.min_val}, {var.max_val}]"  
            info_text += "\\n"  
          
        ax3.text(0.1, 0.9, info_text, va='top', ha='left', transform=ax3.transAxes,  
                fontfamily='monospace')  
          
        plt.tight_layout()  
          
        # Convert to SVG  
        svg_io = io.StringIO()  
        fig.savefig(svg_io, format='svg')  
        plt.close(fig)  
          
        svg_str = svg_io.getvalue()  
          
        # Fix the SVG namespace  
        svg_str = svg_str.replace('xmlns:svg', 'xmlns')  
          
        return svg_str

In [4]:
class LHSApp:
    """Main application class for the LHS Experimental Design tool"""
    
    def __init__(self):
        self.lhs_design = LHSExperimentalDesign()
        self.build_ui()
        
    def build_ui(self):
        """Build the user interface"""
        # Create tabs for different sections
        self.tabs = widgets.Tab()
        
        # Tab 1: Variable Definition
        self.var_tab = self.build_variable_tab()
        
        # Tab 2: Sampling
        self.sampling_tab = self.build_sampling_tab()
        
        # Tab 3: Results
        self.results_tab = self.build_results_tab()
        
        # Tab 4: Save/Load
        self.save_load_tab = self.build_save_load_tab()
        
        # Set tab titles
        self.tabs.children = [self.var_tab, self.sampling_tab, self.results_tab, self.save_load_tab]
        self.tabs.set_title(0, 'Define Variables')
        self.tabs.set_title(1, 'Generate Samples')
        self.tabs.set_title(2, 'Results')
        self.tabs.set_title(3, 'Save/Load')
        
        # Display the UI
        display(self.tabs)
        
    def build_variable_tab(self):
        """Build the variable definition tab"""
        # Variable type selection
        self.var_type = widgets.Dropdown(
            options=[('Continuous', 'continuous'), ('Discrete', 'discrete'), ('Categorical', 'categorical')],
            value='continuous',
            description='Type:',
            style={'description_width': 'initial'}
        )
        
        # Variable name
        self.var_name = widgets.Text(
            value='Variable 1',
            description='Name:',
            style={'description_width': 'initial'}
        )
        
        # Min, max, step for continuous/discrete variables
        self.var_min = widgets.FloatText(
            value=0.0,
            description='Min:',
            style={'description_width': 'initial'}
        )
        
        self.var_max = widgets.FloatText(
            value=1.0,
            description='Max:',
            style={'description_width': 'initial'}
        )
        
        self.var_step = widgets.FloatText(
            value=0.1,
            description='Step:',
            style={'description_width': 'initial'},
            disabled=True  # Only enabled for discrete variables
        )
        
        # Weight for sampling
        self.var_weight = widgets.FloatSlider(
            value=1.0,
            min=0.1,
            max=5.0,
            step=0.1,
            description='Weight:',
            style={'description_width': 'initial'},
            tooltip='Weight > 1 concentrates points toward lower values, < 1 toward higher values'
        )
        
        # Categories for categorical variables
        self.var_categories = widgets.Text(
            value='A, B, C',
            description='Categories:',
            style={'description_width': 'initial'},
            disabled=True  # Only enabled for categorical variables
        )
        
        # Add variable button
        self.add_var_btn = widgets.Button(
            description='Add Variable',
            button_style='success',
            icon='plus'
        )
        self.add_var_btn.on_click(self.add_variable)
        
        # Variable list
        self.var_list = widgets.SelectMultiple(
            options=[],
            description='Variables:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100%', height='150px')
        )
        
        # Remove variable button
        self.remove_var_btn = widgets.Button(
            description='Remove Selected',
            button_style='danger',
            icon='trash'
        )
        self.remove_var_btn.on_click(self.remove_variable)
        
        # Update UI based on variable type
        def on_var_type_change(change):
            if change['new'] == 'continuous':
                self.var_step.disabled = True
                self.var_categories.disabled = True
                self.var_min.disabled = False
                self.var_max.disabled = False
                self.var_weight.disabled = False
            elif change['new'] == 'discrete':
                self.var_step.disabled = False
                self.var_categories.disabled = True
                self.var_min.disabled = False
                self.var_max.disabled = False
                self.var_weight.disabled = False
            elif change['new'] == 'categorical':
                self.var_step.disabled = True
                self.var_categories.disabled = False
                self.var_min.disabled = True
                self.var_max.disabled = True
                self.var_weight.disabled = True
        
        self.var_type.observe(on_var_type_change, names='value')
        
        # Layout
        var_def_box = widgets.VBox([
            widgets.HBox([self.var_name, self.var_type]),
            widgets.HBox([self.var_min, self.var_max, self.var_step]),
            widgets.HBox([self.var_weight, self.var_categories]),
            self.add_var_btn
        ])
        
        var_list_box = widgets.VBox([
            widgets.Label('Defined Variables:'),
            self.var_list,
            self.remove_var_btn
        ])
        
        return widgets.VBox([
            widgets.HTML('<h3>Define Experimental Variables</h3>'),
            widgets.HTML('<p>Define the variables for your experiment. For each variable, specify its type, range, and other properties.</p>'),
            var_def_box,
            widgets.HTML('<hr>'),
            var_list_box
        ])
    
    def build_sampling_tab(self):
        """Build the sampling tab"""
        # Number of samples
        self.n_samples = widgets.IntSlider(
            value=10,
            min=5,
            max=100,
            step=1,
            description='Number of samples:',
            style={'description_width': 'initial'}
        )
        
        # Random seed
        self.random_seed = widgets.IntText(
            value=42,
            description='Random seed:',
            style={'description_width': 'initial'}
        )
        
        # Generate samples button
        self.generate_btn = widgets.Button(
            description='Generate Samples',
            button_style='primary',
            icon='random'
        )
        self.generate_btn.on_click(self.generate_samples)
        
        # Output area for messages
        self.sampling_output = widgets.Output()
        
        return widgets.VBox([
            widgets.HTML('<h3>Generate Experimental Design</h3>'),
            widgets.HTML('<p>Specify the number of experiments to generate and other sampling parameters.</p>'),
            widgets.HBox([self.n_samples, self.random_seed]),
            self.generate_btn,
            widgets.HTML('<hr>'),
            self.sampling_output
        ])
    
    def build_results_tab(self):
        """Build the results tab"""
        # Experiment table
        self.results_output = widgets.Output()
        
        # Visualization options
        self.viz_type = widgets.Dropdown(
            options=[
                ('Pairwise Scatter Plots', 'scatter'),
                ('Distributions', 'dist'),
                ('Parallel Coordinates', 'parallel')
            ],
            value='scatter',
            description='Visualization:',
            style={'description_width': 'initial'}
        )
        
        # Plot button
        self.plot_btn = widgets.Button(
            description='Generate Plot',
            button_style='info',
            icon='chart-line'
        )
        self.plot_btn.on_click(self.generate_plot)
        
        # Plot output
        self.plot_output = widgets.Output()
        
        # Export button
        self.export_btn = widgets.Button(
            description='Export to CSV',
            button_style='success',
            icon='download'
        )
        self.export_btn.on_click(self.export_to_csv)
        
        # Export filename
        self.export_filename = widgets.Text(
            value='experiments.csv',
            description='Filename:',
            style={'description_width': 'initial'}
        )
        
        return widgets.VBox([
            widgets.HTML('<h3>Experimental Design Results</h3>'),
            widgets.HTML('<p>View and analyze the generated experimental design.</p>'),
            self.results_output,
            widgets.HTML('<hr>'),
            widgets.HTML('<h4>Visualize Design Space</h4>'),
            widgets.HBox([self.viz_type, self.plot_btn]),
            self.plot_output,
            widgets.HTML('<hr>'),
            widgets.HTML('<h4>Export Results</h4>'),
            widgets.HBox([self.export_filename, self.export_btn])
        ])
    
    def build_save_load_tab(self):
        """Build the save/load tab"""
        # Save configuration
        self.save_filename = widgets.Text(
            value='lhs_config.json',
            description='Save filename:',
            style={'description_width': 'initial'}
        )
        
        self.save_btn = widgets.Button(
            description='Save Configuration',
            button_style='primary',
            icon='save'
        )
        self.save_btn.on_click(self.save_configuration)
        
        # Load configuration
        self.load_filename = widgets.Text(
            value='lhs_config.json',
            description='Load filename:',
            style={'description_width': 'initial'}
        )
        
        self.load_btn = widgets.Button(
            description='Load Configuration',
            button_style='info',
            icon='upload'
        )
        self.load_btn.on_click(self.load_configuration)
        
        # Output area for messages
        self.save_load_output = widgets.Output()
        
        return widgets.VBox([
            widgets.HTML('<h3>Save and Load Configurations</h3>'),
            widgets.HTML('<p>Save your current variable configuration or load a previously saved one.</p>'),
            widgets.HBox([self.save_filename, self.save_btn]),
            widgets.HBox([self.load_filename, self.load_btn]),
            self.save_load_output
        ])
    
    def add_variable(self, b):
        """Add a variable to the experimental design"""
        var_type = self.var_type.value
        name = self.var_name.value
        
        # Validate input
        if not name:
            with self.sampling_output:
                clear_output()
                print("Error: Variable name cannot be empty.")
            return
        
        try:
            # Create variable based on type
            if var_type == 'categorical':
                # Parse categories
                categories_str = self.var_categories.value
                if not categories_str:
                    with self.sampling_output:
                        clear_output()
                        print("Error: Categories cannot be empty for categorical variables.")
                    return
                
                categories = [c.strip() for c in categories_str.split(',')]
                var = Variable(name, var_type, categories=categories)
            else:
                min_val = self.var_min.value
                max_val = self.var_max.value
                
                if min_val >= max_val:
                    with self.sampling_output:
                        clear_output()
                        print("Error: Min value must be less than max value.")
                    return
                
                step = None
                if var_type == 'discrete':
                    step = self.var_step.value
                    if step <= 0:
                        with self.sampling_output:
                            clear_output()
                            print("Error: Step must be greater than 0 for discrete variables.")
                        return
                
                weight = self.var_weight.value if var_type != 'categorical' else 1.0
                var = Variable(name, var_type, min_val, max_val, step, weight=weight)
            
            # Add variable to the design
            self.lhs_design.add_variable(var)
            
            # Update variable list
            self.update_variable_list()
            
            # Clear output
            with self.sampling_output:
                clear_output()
                print(f"Added variable: {name}")
            
            # Increment variable name for convenience
            try:
                # Try to extract number from the end of the name
                base_name = ''.join([c for c in name if not c.isdigit()])
                num = int(''.join([c for c in name if c.isdigit()] or '0')) + 1
                self.var_name.value = f"{base_name}{num}"
            except:
                # If that fails, just append a number
                self.var_name.value = f"{name} 2"
                
        except Exception as e:
            with self.sampling_output:
                clear_output()
                print(f"Error adding variable: {str(e)}")
    
    def remove_variable(self, b):
        """Remove selected variables from the experimental design"""
        selected = self.var_list.index
        if not selected:
            with self.sampling_output:
                clear_output()
                print("No variables selected for removal.")
            return
        
        # Remove in reverse order to avoid index shifting
        for idx in sorted(selected, reverse=True):
            self.lhs_design.remove_variable(idx)
        
        # Update variable list
        self.update_variable_list()
        
        with self.sampling_output:
            clear_output()
            print(f"Removed {len(selected)} variable(s).")
    
    def update_variable_list(self):
        """Update the variable list display"""
        var_strings = []
        for var in self.lhs_design.variables:
            if var.var_type == 'categorical':
                var_str = f"{var.name} (Categorical: {', '.join(var.categories[:3])}{'...' if len(var.categories) > 3 else ''})"
            else:
                var_str = f"{var.name} ({var.var_type.capitalize()}: {var.min_val} to {var.max_val})"
                if var.var_type == 'discrete' and var.step:
                    var_str += f", step={var.step}"
                if var.weight != 1.0:
                    var_str += f", weight={var.weight:.1f}"
            var_strings.append(var_str)
        
        self.var_list.options = var_strings
    
    def generate_samples(self, b):
        """Generate samples using LHS"""
        if not self.lhs_design.variables:
            with self.sampling_output:
                clear_output()
                print("Error: No variables defined. Please add variables first.")
            return
        
        n_samples = self.n_samples.value
        seed = self.random_seed.value
        
        try:
            # Generate samples
            df = self.lhs_design.generate_samples(n_samples, seed)
            
            # Display results
            with self.sampling_output:
                clear_output()
                print(f"Generated {n_samples} samples using Latin Hypercube Sampling.")
            
            with self.results_output:
                clear_output()
                display(HTML(f"<h4>Experimental Design ({n_samples} experiments)</h4>"))
                display(df)
            
            # Switch to results tab
            self.tabs.selected_index = 2
        except Exception as e:
            with self.sampling_output:
                clear_output()
                print(f"Error generating samples: {str(e)}")
    
    def generate_plot(self, b):
        """Generate visualization of the samples"""
        if self.lhs_design.experiment_df is None:
            with self.plot_output:
                clear_output()
                print("Error: No samples generated yet. Please generate samples first.")
            return
        
        viz_type = self.viz_type.value
        
        with self.plot_output:
            clear_output()
            
            try:
                if viz_type == 'scatter':
                    fig = self.lhs_design.plot_samples()
                    if fig is None:
                        print("Error: Not enough continuous/discrete variables for scatter plots.")
                        return
                elif viz_type == 'dist':
                    fig = self.lhs_design.plot_distributions()
                    if fig is None:
                        print("Error: No continuous/discrete variables for distribution plots.")
                        return
                elif viz_type == 'parallel':
                    fig = self.lhs_design.plot_parallel_coordinates()
                    if fig is None:
                        print("Error: Not enough variables for parallel coordinates plot.")
                        return
                
                plt.tight_layout()
                display(fig)
            except Exception as e:
                print(f"Error generating plot: {str(e)}")
    
    def export_to_csv(self, b):
        """Export the experimental design to a CSV file"""
        if self.lhs_design.experiment_df is None:
            with self.plot_output:
                clear_output()
                print("Error: No samples generated yet. Please generate samples first.")
            return
        
        filename = self.export_filename.value
        if not filename.endswith('.csv'):
            filename += '.csv'
        
        success = self.lhs_design.save_to_csv(filename)
        
        with self.plot_output:
            clear_output()
            if success:
                print(f"Experimental design saved to {filename}")
            else:
                print("Error: Failed to save experimental design.")
    
    def save_configuration(self, b):
        """Save the current configuration to a JSON file"""
        if not self.lhs_design.variables:
            with self.save_load_output:
                clear_output()
                print("Error: No variables defined. Please add variables first.")
            return
        
        filename = self.save_filename.value
        if not filename.endswith('.json'):
            filename += '.json'
        
        success = self.lhs_design.save_configuration(filename)
        
        with self.save_load_output:
            clear_output()
            if success:
                print(f"Configuration saved to {filename}")
            else:
                print("Error: Failed to save configuration.")
    
    def load_configuration(self, b):
        """Load a configuration from a JSON file"""
        filename = self.load_filename.value
        
        success = self.lhs_design.load_configuration(filename)
        
        with self.save_load_output:
            clear_output()
            if success:
                print(f"Configuration loaded from {filename}")
                self.update_variable_list()
            else:
                print(f"Error: Failed to load configuration from {filename}")

# Create and display the app
app = LHSApp()

## How to Use This Application

### 1. Define Variables
- Start by defining your experimental variables in the "Define Variables" tab
- For each variable, specify:
  - Name: A descriptive name for the variable
  - Type: Continuous (any value in a range), Discrete (stepped values), or Categorical (specific options)
  - Range: Min and max values for continuous/discrete variables
  - Step: For discrete variables, the increment between values
  - Categories: For categorical variables, comma-separated list of options
  - Weight: For continuous/discrete variables, controls sampling distribution (>1 focuses on lower values, <1 on higher values)

### 2. Generate Samples
- In the "Generate Samples" tab, specify the number of experiments you want to generate
- Set a random seed for reproducibility
- Click "Generate Samples" to create your experimental design

### 3. View and Analyze Results
- The "Results" tab shows your generated experimental design as a table
- Visualize the design space using different plot types:
  - Pairwise Scatter Plots: Shows the distribution of samples across pairs of variables
  - Distributions: Shows histograms of each variable to verify sampling distribution
  - Parallel Coordinates: Shows all variables together to identify patterns
- Export your experimental design to a CSV file for use in your lab

### 4. Save and Load Configurations
- Save your variable configuration to reuse later
- Load previously saved configurations

## Tips for Effective Experimental Design

- **Variable Weights**: Use weights to focus sampling in regions of interest. For example, if lower temperatures are more promising, use a weight > 1 for temperature.
- **Number of Samples**: Start with 10-15 samples for initial screening, then add more targeted experiments based on results.
- **Visualization**: Use the parallel coordinates plot to identify patterns across all variables simultaneously.
- **Iterative Approach**: After running initial experiments, you can generate additional samples with refined ranges based on promising results.

## Running with Voila

To run this application as a standalone web application:

1. Install Voila: `pip install voila`
2. Run: `voila lhs_app.ipynb`

This will open the application in your web browser as an interactive dashboard.